<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Prompts_e_Ingenier%C3%ADa_de_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompts e Ingeniería de Prompts dentro de LangChain  
---
Este cuaderno se enfoca en los **Prompts** y en la **Ingeniería de Prompts** dentro de LangChain. Abordaremos los fundamentos de los prompts, por qué son tan importantes y cómo LangChain nos ayuda a crearlos y reutilizarlos de manera efectiva con distintas clases de Templates: `PromptTemplate`, `FewShotPromptTemplate` y `ChatPromptTemplate`.

## 1. Introducción a los Prompts

En el contexto de los grandes modelos de lenguaje (Large Language Models - LLMs), el prompt es el mensaje, instrucción o pregunta que proporcionamos para guiar la respuesta del modelo, por ejemplo:

-  “Explícame en tres frases qué es el *Machine Learning*”.

Cuando diseñamos prompts, buscamos que el modelo entienda el contexto y la tarea específica que queremos resolver.

Un buen prompt permite orientar al LLM para que genere una respuesta más precisa y alineada con la tarea.  

Mediante técnicas como *few-shot prompting*, *Chain-of-Thought (CoT) Prompting* y otras, podemos moldear la personalidad y estilo de la respuesta.  

Los prompts son fundamentales en el manejo de modelos de lenguaje ya que guían al modelo para generar respuestas coherentes y útiles. Un prompt bien diseñado puede marcar la diferencia entre una salida precisa y relevante, y una respuesta confusa o fuera de contexto.  

LangChain ofrece herramientas específicas para gestionar y optimizar prompts de manera eficiente. Estas herramientas permiten crear plantillas de prompts reutilizables, adaptarlas dinámicamente según el contexto y personalizarlas para diferentes aplicaciones.


## 2. Configuración del entorno del cuaderno

Instalamos los paquetes necesarios para LangChain, incluyendo soporte para OpenAI y Groq, y recupera las claves API correspondientes desde los secretos de Colab para fines de autenticación.
(Este codigo se explico con detalle en el cuaderno anterior)



In [ ]:
!pip install langchain -qU

!pip install langchain-openai -qU
!pip install langchain-groq -qU
# !pip install langchain-google-genai -qU
# !pip install langchain-huggingface -qU

from google.colab import userdata

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')


## 3. PromptTemplate
---
Los prompt templates en Langchain permiten crear prompts dinámicos y flexibles mediante la incorporación de variables y opciones de formato, lo que le permite personalizar los prompts en función de los datos de entrada o tareas específicas.

- Ayudan en la construccion de prompts complejos.
- Son dinamicos: tienen *placeholders* para variables {variable}  
- Pueden reutilizarse  

Imagina que quieres reutilizar la misma estructura de prompt para múltiples entradas; en lugar de escribir el prompt completo cada vez, puedes definir una “plantilla” con huecos que se llenan de forma programática.

Ejemplos:

- "Explicame {tema} con nivel de dificultad {nivel}"

- "Traduce {frase} del {idioma_entrada} a {idioma_salida}"

Prompts como los anteriores son genericos y reutilizables, pues se usan com plantillas para la construcción de prompts concretos.

### Sintaxis básica

`PromptTemplate` es la clase base que nos permite crear plantillas de prompts con variables dinámicas.



```
prompt_template = PromptTemplate(
            input_variables=["variable1", "variable2"],  # Lista de variables dinámicas
            template="Este es un ejemplo donde {variable1} y {variable2} son variables dinámicas."
)
```



*Observación: De momento no vamos a consultara ningún LLM. Sólo estamos construyendo un prompt que luego podriamos pasar a un modelo.*

1. Definimos una cadena de texto (**`plantilla_texto`**) con dos variables dinámicas: `{tema}` y `{pregunta}`.

2. Usamos el contructor de clase `PromptTemplate()` para crear un objeto PromptTemplate indicando cuáles son esas variables (input_variables) y cuál es la plantilla (template).

3. Luego usamos el metodo `.format()` de la clase, para rellenar los huecos del template.

In [ ]:
from langchain import PromptTemplate

plantilla_texto = """
Eres un experto en {tema}.
Por favor, responde la siguiente pregunta con detalle:
Pregunta: {pregunta}
"""

prompt_template = PromptTemplate(
    input_variables=["tema", "pregunta"],
    template=plantilla_texto
)

prompt=prompt_template.format(tema="programacion", pregunta="Que es python?")

prompt

La clase `PromptTemplate` tiene un metodo que permite instanciar objetos sin necesidad de usar el contructor de clase lo que permite que el codigo sea mas compacto y legible. Este metodo es `.from_template()`


```
prompt_template = "Hola, {nombre}. Hoy es {dia}. ¿Cómo estás?"
prompt = PromptTemplate.from_template(prompt_template)
```



`PromptTemplate.from_template()` es un _método de clase_ que permite crear una instancia de `PromptTemplate` de manera directa a partir de una cadena de texto que contiene variables de relleno (por ejemplo, `{tema}` y `{pregunta}`). A diferencia de usar el constructor estándar (donde se debe especificar `input_variables` y la plantilla por separado), esta función:

1. **Extrae automáticamente** los nombres de las variables desde las llaves (p. ej., `{tema}`, `{pregunta}`).
2. **Crea y retorna** un objeto `PromptTemplate` listo para usar, lo que hace el código más compacto y legible.   

Este es el metodo preferido para crear PromptTemplates

In [ ]:
from langchain_core.prompts import PromptTemplate

#Instanciar usando .from_template es la forma preferida
prompt_template = PromptTemplate.from_template("""
Eres un experto en {tema}.
Por favor, responde la siguiente pregunta con detalle:
Pregunta: {pregunta}
""")
prompt = prompt_template.format(tema="programación", pregunta="Que es python?")

prompt

Vamos ahora a crear un prompt template y enviarselo a un LLM

1.   Elemento de lista
2.   Elemento de lista



In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

llm1 = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY)
llm2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

plantilla_texto = """
Eres un experto en traducción del {idioma_origen} al inglés.
Por favor, traduce el siguiente texto:
---
Texto: {texto_a_traducir}
---
Mantén la intención, el estilo y el tono original.
Proporciona solo el texto traducido, sin comentarios adicionales
"""

# Creamos el PromptTemplate usando las variables definidas
prompt_template = PromptTemplate(
    input_variables=["idioma_origen", "texto_a_traducir"],
    template=plantilla_texto
)

# Ejemplo de uso
prompt = prompt_template.format(
    idioma_origen="español",
    texto_a_traducir="¡Hola! ¿Cómo te va?"
)


#Invocamos los modelos pasandoles el prompt
respuesta1 = llm1.invoke(prompt)
respuesta2 = llm2.invoke(prompt)


# Extraemos de la respuesta el nombre del modelo y el resultado
print(respuesta1.response_metadata["model_name"]) # imprimimos el nombre del modelo
print(respuesta1.content)
print("---")
print(respuesta2.response_metadata["model_name"]) # imprimimos el nombre del modelo
print(respuesta2.content)


### Reto 1: Crea tu propio PromptTemplate

**Objetivo**  

Diseñar un `PromptTemplate` que reciba variables para traducir un texto de un idioma a otro, especificando además el estilo de traducción. Luego, pasar este prompt a un LLM para obtener el resultado y verificar su eficacia.

**Instrucciones**

1. **Crea un PromptTemplate** con **tres** variables:
    
    - `{idioma_origen}`: El idioma original en el que está el texto.
    - `{idioma_destino}`: El idioma al que se va a traducir.
    - `{texto_original}`: El texto que se quiere traducir.
2. **Incluye** en tu plantilla alguna instrucción sobre el **estilo** o **tono** de la traducción. Por ejemplo, podrías pedir que sea “formal” o “informal”.
    
3. **Crea** el prompt final usando `.format(...)` y pásaselo a un LLM de tu elección. Obtén la respuesta y **muéstrala** en pantalla.
    
4. (Opcional) **Experimenta** con distintos valores de _temperature_ o _top\_p_ (dependiendo del LLM que uses) para comparar la **consistencia** de la traducción.

In [ ]:
# Escribe aquí tu solución para el reto 1

### Solución Reto 1

In [ ]:
# Realmente no es necesario importar de nuevo las librerías.
# Solo lo hacemos con fines ilustrativos, para que el ejemplo sea autosuficiente.

from langchain import PromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# 1. Define la plantilla con las variables {idioma_origen}, {idioma_destino}, y {texto_original}
mi_plantilla = """
Eres un traductor experto. Traducirás cuidadosamente del {idioma_origen} al {idioma_destino}.
Por favor, mantén un tono formal en la traducción.
---
Texto original: {texto_original}
---
"""

# 2. Crea el PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["idioma_origen", "idioma_destino", "texto_original"],
    template=mi_plantilla
)

# 3. Usa .format para rellenar los huecos
prompt = prompt_template.format(
    idioma_origen="español",
    idioma_destino="francés",
    texto_original="¡Hola! ¿Cómo estás hoy?"
)

# 4. Invoca tu LLM y muestra el resultado
# (Ejemplo con un LLM hipotético `my_llm`)
response = llm.invoke(prompt, temperature=0.5)

print("Prompt final:\n", prompt)
print("\nTraducción:\n", response.content)


## 4. FewShotPromptTemplate: proporcionando ejemplos de referencia

**Few-shot prompting** es una técnica donde proporcionamos ejemplos dentro del prompt para que el modelo entienda mejor el contexto y la forma de la respuesta. Con `FewShotPromptTemplate` podemos manejar varios ejemplos y luego añadir la pregunta final.

### Sintaxis



```
FewShotPromptTemplate(
    examples=ejemplos,               # Lista de ejemplos
    example_prompt=example_prompt,   # PromptTemplate para cada ejemplo
    prefix=prefix_text,              # Texto que aparecerá antes de los ejemplos
    suffix=suffix_text,              # Texto que aparecerá después de los ejemplos, habitualmente la pregunta
    input_variables=["nuevo_input"], # Variables adicionales que usarás
    example_separator="\n---\n"      # Separador entre ejemplos
)
```



### ¿Cómo funciona FewShotPromptTemplate?

**¿Cómo funciona `FewShotPromptTemplate`?**

1. **`examples`:** Una lista de diccionarios. Cada diccionario representa un ejemplo y contiene pares clave-valor con las entradas y salidas deseadas para una tarea específica.
    
2. **`example_prompt`:** Una instancia de `PromptTemplate` que define cómo se formatea cada ejemplo individualmente. Define las variables de entrada que se utilizarán para cada ejemplo.
    
3. **`prefix`:** Una cadena que va al principio del prompt completo. Suele ser una instrucción general o una descripción de la tarea.
    
4. **`suffix`:** Una cadena que va al final del prompt completo. Normalmente aquí es donde se introduce la pregunta al modelo, que debria ser como los ejemplos, pero sin el resultado
    
5. **`input_variables`:** Una lista de nombres de variables que se utilizarán en el `suffix`.
    
6. **`example_separator`:** Una cadena que separa cada ejemplo en el prompt. Por defecto es `\n\n`.
    
    

**En resumen, `FewShotPromptTemplate` toma los ejemplos, los formatea usando `example_prompt`, los une con `example_separator`, y los coloca entre `prefix` y `suffix` para crear el prompt final. Este prompt contiene el contexto necesario (los ejemplos) para que el modelo de lenguaje pueda inferir la tarea y aplicarla a la nueva entrada proporcionada en el `suffix`.**

Hay que usar la sintaxis con el constructor de clase. No hay disponible ningun metodo que simplifique la sintaxis.

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# Lista de ejemplo, donde cada ejemplo es un diccionario
ejemplos = [
    {"input": "Hello", "output": "Bonjour"},
    {"input": "Goodbye", "output": "Au revoir"},
    {"input": "Thank you", "output": "Merci"},
]

#Instancia de PromptTemplate para formatear los ejemplos
traduccion_prompt_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Creamos el prompt pasando:
# la lista de ejemplos
# el formateador de los ejemplos
# el prefix, que suele ser la descripcion general de la tarea
# el suffix, que donde introduciomes la pregunta con la variables
# La lista de variables


prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=traduccion_prompt_template,
    prefix="Translate the following English words to French:",
    suffix="Input: {english_word}\nOutput:",
    input_variables=["english_word"],
)

print(prompt.format(english_word="Car"))

Vamos a usarlo para realizar una llamada a los modelos que estamos usando en este cuaderno

In [ ]:
from langchain import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Creamos las instancias de cada modelo.
llm1 = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm2 = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# 1) Definimos algunos ejemplos de traducción: (input → output)
ejemplos = [
    {"input": "Buenos días", "output": "Good morning"},
    {"input": "¿Dónde está el baño?", "output": "Where is the bathroom?"}
]

# 2) Creamos un PromptTemplate para cada ejemplo: cómo se mostrará "input" y "output" en el prompt
traduccion_prompt_template = PromptTemplate(
    input_variables=["input", "output"],
    template="""
Ejemplo de Entrada: {input}
Ejemplo de Salida: {output}
"""
)

# 3) Definimos un prefijo y un sufijo para contextualizar la tarea antes y después de los ejemplos
prefix_text = """Eres un traductor experto del español al inglés.
Estos son algunos ejemplos de cómo traducir oraciones al inglés:"""

suffix_text = """
Ahora traduce la siguiente frase:
Entrada: {nueva_frase}
Salida:
"""

# 4) Creamos el FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=ejemplos,                  # Nuestros ejemplos
    example_prompt=traduccion_prompt_template,      # Formato de cada ejemplo
    prefix=prefix_text,                 # Texto antes de los ejemplos
    suffix=suffix_text,                 # Texto después de los ejemplos
    input_variables=["nueva_frase"],    # Variable requerida al formatear
    example_separator="\n---\n"         # Separador entre ejemplos
)

# 5) Generamos el prompt final para la nueva frase
prompt = few_shot_prompt.format(nueva_frase="¡Hola! ¿Cómo te va?")

print("=== Prompt Generado ===")
print(prompt)

# 6) Pasamos el prompt al primer modelo (ChatOpenAI - gpt-4o-mini)
print("\n=== Respuesta con ChatOpenAI (gpt-4o-mini) ===")
response_llm1 = llm1.invoke(prompt)
print(response_llm1.content)

# 7) Pasamos el mismo prompt al segundo modelo (ChatGroq - llama-3.3-70b-versatile)
print("\n=== Respuesta con ChatGroq (llama-3.3-70b-versatile) ===")
response_llm2 = llm2.invoke(prompt)
print(response_llm2.content)


## Reto 2: Few-Shot Prompting para Traducción Creativa

El reto consiste en usar la clase FewShotPromptTemplate para traducir oraciones del español al inglés con un estilo específico, incorporando ejemplos previos que sirvan de guía para el LLM.  

### **Instrucciones**

1. **Crea una lista de ejemplos** de traducción que contengan:
    
    - La frase de entrada en español (clave: `"input"`).
    - Su traducción al inglés, **pero con cierto estilo o “sabor”** (clave: `"output"`).
        - Por ejemplo, podrías elegir que la traducción sea **muy “entusiasta”** o **tenga un tono humorístico**.
2. **Diseña un `PromptTemplate`** para mostrar cada ejemplo con un formato claro. Algo así como:
    
    python
    
    Copiar código
    
    `"Oración original: {input}\nTraducción estilizada: {output}\n"`
    
    (puedes ajustar el texto a tu preferencia).
    
3. **Define** el `prefix` y el `suffix` de tu `FewShotPromptTemplate`:
    
    - El `prefix` debe _explicar_ que el LLM es un traductor con cierto estilo (por ejemplo, “traductor humorístico” o “traductor formal”).
    - El `suffix` debe contener la **nueva frase** a traducir (`{frase_nueva}`) y especificar que se aplique el mismo estilo.
4. **Crea** la instancia de `FewShotPromptTemplate` con:
    
    - `examples`: tu lista de ejemplos.
    - `example_prompt`: tu plantilla de ejemplo.
    - `prefix`, `suffix`, `example_separator`, etc.
5. **Usa `.format()`** para generar el prompt final, pasando la nueva frase a traducir en la variable `{frase_nueva}`.
    
6. **Envía** ese prompt a un LLM (puede ser `ChatOpenAI`, `ChatGroq` o cualquier otro). Muestra la respuesta que obtienes.
    
7. (Opcional) **Experimenta** con distintos estilos (más formal, más bromista, etc.) para ver cómo cambian las traducciones.

8. (Opcional). Puedes incluir el estilo de traducción como una varibale dinámica del prompt ?

## Solución Reto 2

In [ ]:
from langchain import FewShotPromptTemplate, PromptTemplate

# 1. Ejemplos
ejemplos = [
    {
        "input": "Buenos días, ¿cómo amaneciste?",
        "output": "Good morning, how did you wake up, buddy? (super friendly tone)"
    },
    {
        "input": "¿Quieres salir a correr más tarde?",
        "output": "Do you want to go for a run later, pal? (energetic style)"
    }
]

# 2. Plantilla de ejemplo
ejemplo_plantilla = """
Oración original: {input}
Traducción con estilo: {output}
"""

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=ejemplo_plantilla
)

# 3. Prefijo y Sufijo
prefix_text = """Eres un traductor que siempre usa un tono muy amigable y entusiasta.
Aquí tienes algunos ejemplos de cómo traduces del español al inglés:
"""
suffix_text = """
Ahora quiero que traduzcas la siguiente frase con el mismo tono amistoso:
Oración: {frase_nueva}
Traducción:
"""

# 4. FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=example_prompt,
    prefix=prefix_text,
    suffix=suffix_text,
    input_variables=["frase_nueva"],
    example_separator="\n---\n"
)

# 5. Formateo del prompt
prompt_final = few_shot_prompt.format(frase_nueva="¡Hola! ¿Listo para la aventura de hoy?")

# 6. (Pseudo-código) Llamada al LLM
# response = llm.invoke(prompt_final)
# print(response.content)





---



---



In [ ]:
# Para eviar el uso de algunas variables, se suele escribir de forma mas compacta
# en adelante usaremos esta forma.

prompt_template = PromptTemplate.from_template("Cuentame un chiste sobre {tema}")
prompt = prompt_template.format(tema="enanos")

prompt

In [ ]:
#Instanciar usando .from_template es la forma preferida
prompt_template = PromptTemplate.from_template("Cuentame un chiste {adjetivo} sobre {tema}")
prompt = prompt_template.format(adjetivo="malo", tema="enanos")

prompt

In [ ]:
# Usando el constructor de clase
prompt_template = PromptTemplate(
    input_variables=["adjetivo", "tema"],
    template="Cuentame un chiste {adjetivo} sobre {tema}")

prompt = prompt_template.format(adjetivo="malo", tema="enanos")

prompt


In [ ]:
prompt_template = PromptTemplate.from_template('''
Proporciona un resumen conciso del siguiente tema: {tema}
Usa un nivel de lenguaje {nivel}
Incluye al menos los siguinetes puntos clave:
* Información de antecedentes
* Ideas principales
* Detalles de apoyo
* Importancia o impacto
Manten una actitud neutral y objetiva en tu resumen.
''')
prompt = prompt_template.format(tema="Cambio climatico", nivel="técnico")

prompt

In [ ]:
diagnostico_prompt_template = PromptTemplate.from_template('''
Eres un médico informado y empático. Con base en los siguientes síntomas descritos por un paciente, proporciona un diagnóstico potencial y sugiere los próximos pasos para el tratamiento.
Síntomas del paciente: {síntomas}
''')

prompt=diagnostico_prompt_template.format(síntomas="Fiebre, dolor de garganta, tos seca")

prompt

In [ ]:
consejo_financiero_prompt_template = PromptTemplate.from_template('''
Eres un asesor financiero certificado. En función del perfil financiero del usuario, brindar asesoramiento de inversión personalizado.
Perfil de usuario:
- Edad: {edad}
- Ingresos: {ingresos}
- Tolerancia al riesgo: {tolerancia_riesgo}
- Objetivos de inversión: {objetivos_inversion}
''')

prompt = consejo_financiero_prompt_template.format(edad=50, ingresos="100000€", tolerancia_riesgo="moderada", objetivos_inversion="ahorros jubilación" )

prompt

`FewShotPromptTemplate` en LangChain es una herramienta poderosa para el aprendizaje de pocos ejemplos (few-shot learning) en aplicaciones de lenguaje. Te permite definir un conjunto de ejemplos de entrada/salida y, junto con una plantilla de prompt, generar prompts para un modelo de lenguaje que imitan un comportamiento deseado. Básicamente, ayuda a entrenar a un modelo para que aprenda a partir de unos pocos ejemplos cómo realizar una tarea.

Aquí te explico su funcionamiento y te doy ejemplos:

**¿Cómo funciona `FewShotPromptTemplate`?**

1. **`examples`:** Un array de diccionarios. Cada diccionario representa un ejemplo y contiene pares clave-valor con las entradas y salidas deseadas para una tarea específica.
    
2. **`example_prompt`:** Una instancia de `PromptTemplate` que define cómo se formatea cada ejemplo individualmente. Define las variables de entrada que se utilizarán para cada ejemplo.
    
3. **`prefix`:** Una cadena que va al principio del prompt completo. Suele ser una instrucción general o una descripción de la tarea.
    
4. **`suffix`:** Una cadena que va al final del prompt completo. Normalmente aquí es donde se introduce la nueva entrada a procesar por el modelo.
    
5. **`input_variables`:** Una lista de nombres de variables que se utilizarán en el `suffix`.
    
6. **`example_separator`:** Una cadena que separa cada ejemplo en el prompt. Por defecto es `\n\n`.
    
7. **`template_format`:** Define el formato en el que se va a procesar la plantilla. Por defecto es `f-string`. Otras opciones son `jinja2`.
    

**En resumen, `FewShotPromptTemplate` toma los ejemplos, los formatea usando `example_prompt`, los une con `example_separator`, y los coloca entre `prefix` y `suffix` para crear el prompt final. Este prompt contiene el contexto necesario (los ejemplos) para que el modelo de lenguaje pueda inferir la tarea y aplicarla a la nueva entrada proporcionada en el `suffix`.**

Hay que usar la sintaxis con el constructor de clase. No hay disponible ningun metodo que simplifique la sintaxis.

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# Lista de ejemplo, donde cada ejemplo es un diccionario
ejemplos = [
    {"input": "Hello", "output": "Bonjour"},
    {"input": "Goodbye", "output": "Au revoir"},
    {"input": "Thank you", "output": "Merci"},
]

#Instancia de PromptTemplate para formatear los ejemplos
traduccion_prompt_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Creamos el prompt pasando:
# la lista de ejemplos
# el formateador de los ejemplos
# el prefix, que suele ser la descripcion general de la tarea
# el suffix, que donde introduciomes la pregunta con la variables
# La lista de variables


prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=traduccion_prompt_template,
    prefix="Translate the following English words to French:",
    suffix="Input: {english_word}\nOutput:",
    input_variables=["english_word"],
)

print(prompt.format(english_word="Car"))

Crear un prompt con ejemplos para que indique la capital de un pais

Refs:



Las prompt templatespermiten crear solicitudes dinámicas y flexibles mediante la incorporación de variables y opciones de formato, lo que le permite personalizar las solicitudes en función de los datos de entrada o tareas específicas.

- Son dinamicas: tinene placeholders para variables
- Pueden reutilizarse

Un prompt template como este "Explicame {tema} con nivel de dificultad {nivel}" es reutilizable y flexible, sirve para muchos situaciones.

"traduce {frase} del {idioma_entrada} a {idioma_salida}"


Diferencias entre ChatPromptTemplate y PrompTemplate
Difrenecias entre .format y .invoke

- Pueden tener locga condicional interna (if)
- Pueden anidarse

---
¿Qué es Prompt?  

Un mensaje para un modelo de lenguaje es un conjunto de instrucciones o entradas proporcionadas por un usuario para guiar la respuesta del modelo, ayudándolo a comprender el contexto y generar resultados relevantes y coherentes basados ​​en el lenguaje, como responder preguntas, completar oraciones o participar en una conversación.  

---
3 tipos de plantillas de indicaciones de LangChain¶
Cuando se solicita en LangChain, se recomienda (aunque no es obligatorio) utilizar una clase de plantilla predefinida como:

PromptTemplatepara crear indicaciones básicas.
FewShotPromptTemplatepara un aprendizaje de unos pocos disparos.
ChatPromptTemplatepara modelar interacciones de chatbot.